In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import os
import json

In [ ]:
class WeatherForecastCollector:
    def __init__(self, api_key, locations, output_dir='weather_forecasts'):
        self.api_key = api_key
        self.locations = locations
        self.output_dir = output_dir
        
        os.makedirs(output_dir, exist_ok=True)

    def get_forecast_weather(self, location, start_date):
        base_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
        
        end_date = (datetime.strptime(start_date, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
        
        url = f"{base_url}/{location}/{start_date}/{end_date}?key={self.api_key}&include=hours&unitGroup=metric"
        
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            
            forecast_data = {
                'location': location,
                'forecast_date': start_date,
                'hourly_forecast': []
            }
            
            if 'days' in data and data['days']:
                day_data = data['days'][0]
                
                for hour in day_data.get('hours', []):
                    hourly_forecast = {
                        'datetime': hour.get('datetime'),
                        'temp': hour.get('temp'),
                        'feelslike': hour.get('feelslike'),
                        'humidity': hour.get('humidity'),
                        'windspeed': hour.get('windspeed'),
                        'winddir': hour.get('winddir'),
                        'pressure': hour.get('pressure'),
                        'cloudcover': hour.get('cloudcover'),
                        'conditions': hour.get('conditions'),
                        'description': hour.get('description')
                    }
                    forecast_data['hourly_forecast'].append(hourly_forecast)
                
                return forecast_data
            
        except requests.exceptions.RequestException as e:
            print(f"Помилка запиту для {location} на {start_date}: {e}")
            return None

    def collect_forecasts(self):
        all_forecasts = []
        
        start_date = datetime.now().strftime('%Y-%m-%d')
        
        for location in self.locations:
            forecast = self.get_forecast_weather(location, start_date)
            
            if forecast:
                all_forecasts.append(forecast)
            
            time.sleep(1)
        
        return all_forecasts

    def save_to_csv(self, data):
        if not data:
            print("Немає даних для збереження")
            return
        hourly_data = []
        for forecast in data:
            for hour_forecast in forecast['hourly_forecast']:
                hour_forecast['location'] = forecast['location']
                hour_forecast['forecast_date'] = forecast['forecast_date']
                hourly_data.append(hour_forecast)
        
        df = pd.DataFrame(hourly_data)
        filename = f"{self.output_dir}/forecast_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"Дані збережено в {filename}")

    def save_to_json(self, data):
        if not data:
            print("Немає даних для збереження")
            return
        
        filename = f"{self.output_dir}/forecast_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        print(f"Дані збережено в {filename}")

In [ ]:
def main():
    API_KEY = 'F6BGQLQBVGDVSB3YM6PW25U3S'
    
    locations = ['Lviv,Lviv Oblast', 'Kyiv,Kyiv City', 'Kharkiv,Kharkiv Oblast', 
                 'Odesa,Odesa Oblast', 'Dnipro,Dnipropetrovsk Oblast']
    
    collector = WeatherForecastCollector(API_KEY, locations)
    
    forecast_data = collector.collect_forecasts()
    
    collector.save_to_csv(forecast_data)
    collector.save_to_json(forecast_data)

if __name__ == "__main__":
    main()

Дані збережено в weather_forecasts/forecast_20250327_174129.csv
Дані збережено в weather_forecasts/forecast_20250327_174129.json
